In [1]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.RandomForestClassifier  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoder}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline, PipelineModel}  
import org.apache.log4j._  
Logger.getLogger("org").setLevel(Level.ERROR)

Intitializing Scala interpreter ...

Spark Web UI available at http://atanus-air.cinci.rr.com:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1598634019269)
SparkSession available as 'spark'


import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.log4j._


In [2]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1c5b4d09


In [3]:
val data = spark.read.option("header","true").
    option("inferSchema","true").
    format("csv").
    load("WA_Fn-UseC_-HR-Employee-Attrition.csv")

data: org.apache.spark.sql.DataFrame = [Age: int, Attrition: string ... 33 more fields]


In [4]:
data.show(10)

+---+---------+-----------------+---------+--------------------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalanc

In [6]:
data.columns

res2: Array[String] = Array(Age, Attrition, BusinessTravel, DailyRate, Department, DistanceFromHome, Education, EducationField, EmployeeCount, EmployeeNumber, EnvironmentSatisfaction, Gender, HourlyRate, JobInvolvement, JobLevel, JobRole, JobSatisfaction, MaritalStatus, MonthlyIncome, MonthlyRate, NumCompaniesWorked, Over18, OverTime, PercentSalaryHike, PerformanceRating, RelationshipSatisfaction, StandardHours, StockOptionLevel, TotalWorkingYears, TrainingTimesLastYear, WorkLifeBalance, YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion, YearsWithCurrManager)


In [11]:
data.dtypes

res8: Array[(String, String)] = Array((Age,IntegerType), (Attrition,StringType), (BusinessTravel,StringType), (DailyRate,IntegerType), (Department,StringType), (DistanceFromHome,IntegerType), (Education,IntegerType), (EducationField,StringType), (EmployeeCount,IntegerType), (EmployeeNumber,IntegerType), (EnvironmentSatisfaction,IntegerType), (Gender,StringType), (HourlyRate,IntegerType), (JobInvolvement,IntegerType), (JobLevel,IntegerType), (JobRole,StringType), (JobSatisfaction,IntegerType), (MaritalStatus,StringType), (MonthlyIncome,IntegerType), (MonthlyRate,IntegerType), (NumCompaniesWorked,IntegerType), (Over18,StringType), (OverTime,StringType), (PercentSalaryHike,IntegerType), (PerformanceRating,IntegerType), (RelationshipSatisfaction,IntegerType), (StandardHours,IntegerType), (S...


In [12]:
data.describe()

res9: org.apache.spark.sql.DataFrame = [summary: string, Age: string ... 34 more fields]


In [7]:
data.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- MonthlyRate: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- Over18: string (nullable = true)
 |-- OverTime: string 

In [8]:
data.select("Age").show(10)

+---+
|Age|
+---+
| 41|
| 49|
| 37|
| 33|
| 27|
| 32|
| 59|
| 30|
| 38|
| 36|
+---+
only showing top 10 rows



In [9]:
data.groupBy("Department").count().show()

+--------------------+-----+
|          Department|count|
+--------------------+-----+
|               Sales|  446|
|Research & Develo...|  961|
|     Human Resources|   63|
+--------------------+-----+



In [10]:
data.groupBy("Attrition").count().show()

+---------+-----+
|Attrition|count|
+---------+-----+
|       No| 1233|
|      Yes|  237|
+---------+-----+



In [11]:
import org.apache.spark.sql.functions._
val data2 = data.withColumn( "Attrition", when($"Attrition" === "Yes", 1).otherwise(0))

import org.apache.spark.sql.functions._
data2: org.apache.spark.sql.DataFrame = [Age: int, Attrition: int ... 33 more fields]


In [12]:
data2.groupBy("Attrition").count().show()

+---------+-----+
|Attrition|count|
+---------+-----+
|        1|  237|
|        0| 1233|
+---------+-----+



In [13]:
data2.groupBy("Department").count().show()

+--------------------+-----+
|          Department|count|
+--------------------+-----+
|               Sales|  446|
|Research & Develo...|  961|
|     Human Resources|   63|
+--------------------+-----+



In [14]:
data2.groupBy("Gender").count().show()

+------+-----+
|Gender|count|
+------+-----+
|Female|  588|
|  Male|  882|
+------+-----+



In [15]:
val features = data2.columns.filterNot(_.contains("Attrition"))
val encodedFeatures = features.flatMap{ name =>
    
    val indexer = new StringIndexer()
    .setInputCol(name)
    .setOutputCol(name + "_index")
    
    val oneHotEncoder = new OneHotEncoder()
    .setInputCols(Array(name + "_index"))
    .setOutputCols(Array(name + "_vec"))
    .setDropLast(false)
    
    Array(indexer, oneHotEncoder)
}

val pipeline = new Pipeline().setStages(encodedFeatures)
val indexer_model = pipeline.fit(data2)
val data_trans = indexer_model.transform(data2)
data_trans.show(10)

+---+---------+-----------------+---------+--------------------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+---------+---------------+--------------------+------------------+---------------+-----------------+----------------+--------------+----------------------+--------------------+---------------+-------------+--------------------+------------------+-------------------+-----------------+--------------------+-------------------+-----------------------------+---------------------------+------------+-------------+----------------+---------------+-----------------

| 32|        0|Travel_Frequently|     1005|Research & Develo...|               2|        2| Life Sciences|            1|             8|                      4|  Male|        79|             3|       1|Laboratory Techni...|              4|       Single|         3068|      11864|                 0|     Y|      No|               13|                3|                       3|           80|               0|                8|                    2|              2|             7|                 7|                      3|                   6|      5.0| (43,[5],[1.0])|                 1.0|     (3,[1],[1.0])|           35.0| (886,[35],[1.0])|             0.0| (3,[0],[1.0])|                   0.0|      (29,[0],[1.0])|            2.0|(5,[2],[1.0])|                 0.0|     (6,[0],[1.0])|                0.0|    (1,[0],[1.0])|              1314.0|(1470,[1314],[1.0])|                          1.0|              (4,[1],[1.0])|         0.0|(2,[0],[1.0])|             6.0| (71,[6],[1.0])|                 

| 36|        0|    Travel_Rarely|     1299|Research & Develo...|              27|        3|       Medical|            1|            13|                      3|  Male|        94|             3|       2|Healthcare Repres...|              3|      Married|         5237|      16577|                 6|     Y|      No|               13|                3|                       2|           80|               2|               17|                    3|              2|             7|                 7|                      7|                   7|      3.0| (43,[3],[1.0])|                 0.0|     (3,[0],[1.0])|          523.0|(886,[523],[1.0])|             0.0| (3,[0],[1.0])|                  28.0|     (29,[28],[1.0])|            0.0|(5,[0],[1.0])|                 1.0|     (6,[1],[1.0])|                0.0|    (1,[0],[1.0])|               242.0| (1470,[242],[1.0])|                          0.0|              (4,[0],[1.0])|         0.0|(2,[0],[1.0])|            28.0|(71,[28],[1.0])|                 

features: Array[String] = Array(Age, BusinessTravel, DailyRate, Department, DistanceFromHome, Education, EducationField, EmployeeCount, EmployeeNumber, EnvironmentSatisfaction, Gender, HourlyRate, JobInvolvement, JobLevel, JobRole, JobSatisfaction, MaritalStatus, MonthlyIncome, MonthlyRate, NumCompaniesWorked, Over18, OverTime, PercentSalaryHike, PerformanceRating, RelationshipSatisfaction, StandardHours, StockOptionLevel, TotalWorkingYears, TrainingTimesLastYear, WorkLifeBalance, YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion, YearsWithCurrManager)
encodedFeatures: Array[org.apache.spark.ml.Estimator[_ >: org.apache.spark.ml.feature.OneHotEncoderModel with org.apache.spark.ml.feature.StringIndexerModel <: org.apache.spark.ml.Model[_ >: org.apache.spark.ml.feature.OneHotEnc...


In [16]:
data_trans.groupBy("Gender_index").count().show()

+------------+-----+
|Gender_index|count|
+------------+-----+
|         0.0|  882|
|         1.0|  588|
+------------+-----+



In [17]:
data_trans.groupBy("Department_index").count().show()

+----------------+-----+
|Department_index|count|
+----------------+-----+
|             0.0|  961|
|             1.0|  446|
|             2.0|   63|
+----------------+-----+



In [18]:
data_trans.groupBy("Department_vec").count().show()

+--------------+-----+
|Department_vec|count|
+--------------+-----+
| (3,[2],[1.0])|   63|
| (3,[1],[1.0])|  446|
| (3,[0],[1.0])|  961|
+--------------+-----+



In [19]:
data_trans.groupBy("DistanceFromHome_vec").count().show()

+--------------------+-----+
|DistanceFromHome_vec|count|
+--------------------+-----+
|     (29,[12],[1.0])|   28|
|     (29,[20],[1.0])|   23|
|     (29,[26],[1.0])|   19|
|      (29,[8],[1.0])|   64|
|     (29,[10],[1.0])|   32|
|     (29,[24],[1.0])|   20|
|     (29,[27],[1.0])|   18|
|     (29,[17],[1.0])|   25|
|     (29,[14],[1.0])|   27|
|      (29,[2],[1.0])|   86|
|     (29,[19],[1.0])|   25|
|     (29,[28],[1.0])|   12|
|     (29,[15],[1.0])|   26|
|     (29,[21],[1.0])|   22|
|      (29,[4],[1.0])|   84|
|      (29,[7],[1.0])|   65|
|     (29,[23],[1.0])|   20|
|     (29,[18],[1.0])|   25|
|      (29,[6],[1.0])|   80|
|      (29,[3],[1.0])|   85|
+--------------------+-----+
only showing top 20 rows



In [21]:
data_trans.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Attrition: integer (nullable = false)
 |-- BusinessTravel: string (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- MonthlyRate: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- Over18: string (nullable = true)
 |-- OverTime: strin

In [22]:
val featureCols = Array("Age", "BusinessTravel_vec", "DailyRate", "Department_vec", "DistanceFromHome", "Education", "EducationField_vec", "YearsAtCompany")
val assembler = new VectorAssembler().setInputCols(featureCols).setOutputCol("features")
val data_fea = assembler.transform(data_trans)

featureCols: Array[String] = Array(Age, BusinessTravel_vec, DailyRate, Department_vec, DistanceFromHome, Education, EducationField_vec, YearsAtCompany)
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_e17ad508c418, handleInvalid=error, numInputCols=8
data_fea: org.apache.spark.sql.DataFrame = [Age: int, Attrition: int ... 102 more fields]


In [23]:
data_fea.select("features").show(20)

+--------------------+
|            features|
+--------------------+
|(17,[0,1,4,6,8,9,...|
|(17,[0,2,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,2,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,2,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,2,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,1,4,5,8,9,...|
|(17,[0,3,4,5,8,9,...|
|(17,[0,1,4,6,8,9,...|
|(17,[0,1,4,5,8,9,...|
+--------------------+
only showing top 20 rows



In [24]:
val labelIndexer = new StringIndexer().setInputCol("Attrition").setOutputCol("label")
val data_final = labelIndexer.fit(data_fea).transform(data_fea)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_55d8571889b0
data_final: org.apache.spark.sql.DataFrame = [Age: int, Attrition: int ... 103 more fields]


In [28]:
data_final.select("label", "features").show(20)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(17,[0,1,4,6,8,9,...|
|  0.0|(17,[0,2,4,5,8,9,...|
|  1.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,2,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,2,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,2,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  1.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
|  0.0|(17,[0,3,4,5,8,9,...|
|  0.0|(17,[0,1,4,6,8,9,...|
|  0.0|(17,[0,1,4,5,8,9,...|
+-----+--------------------+
only showing top 20 rows



In [30]:
val Array(training, test) = data_final.select("label","features").
                            randomSplit(Array(0.7, 0.3), seed = 12345)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]


In [31]:
import org.apache.spark.ml.classification.LogisticRegression
val model = new LogisticRegression().fit(training)
val results = model.transform(test).select("features", "label", "prediction")

import org.apache.spark.ml.classification.LogisticRegression
model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_c4ef88e2b3ad, numClasses=2, numFeatures=17
results: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]


In [32]:
val predictionAndLabels = results.
    select($"prediction",$"label").
    as[(Double, Double)].
    rdd

predictionAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[480] at rdd at <console>:44


In [33]:
val bMetrics = new BinaryClassificationMetrics(predictionAndLabels)
val mMetrics = new MulticlassMetrics(predictionAndLabels)
val labels = mMetrics.labels

// Print out the Confusion matrix
println("Confusion matrix:")
println(mMetrics.confusionMatrix)

Confusion matrix:
356.0  0.0  
74.0   3.0  


bMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@71be6e38
mMetrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@4e1d1173
labels: Array[Double] = Array(0.0, 1.0)


In [35]:
// False positive rate by label
labels.foreach { l =>
  println(s"FPR($l) = " + mMetrics.falsePositiveRate(l))
}
// F-measure by label
labels.foreach { l =>
  println(s"F1-Score($l) = " + mMetrics.fMeasure(l))
}

FPR(0.0) = 0.961038961038961
FPR(1.0) = 0.0
F1-Score(0.0) = 0.905852417302799
F1-Score(1.0) = 0.07500000000000001


In [36]:
// Precision by threshold
val precision = bMetrics.precisionByThreshold
precision.foreach { case (t, p) =>
  println(s"Threshold: $t, Precision: $p")
}

// Recall by threshold
val recall = bMetrics.recallByThreshold
recall.foreach { case (t, r) =>
  println(s"Threshold: $t, Recall: $r")
}

// Precision-Recall Curve
val PRC = bMetrics.pr

// F-measure
val f1Score = bMetrics.fMeasureByThreshold
f1Score.foreach { case (t, f) =>
  println(s"Threshold: $t, F-score: $f, Beta = 1")
}

val beta = 0.5
val fScore = bMetrics.fMeasureByThreshold(beta)
f1Score.foreach { case (t, f) =>
  println(s"Threshold: $t, F-score: $f, Beta = 0.5")
}

// AUPRC
val auPRC = bMetrics.areaUnderPR
println("Area under precision-recall curve = " + auPRC)

// Compute thresholds used in ROC and PR curves
val thresholds = precision.map(_._1)

// ROC Curve
val roc = bMetrics.roc

// AUROC
val auROC = bMetrics.areaUnderROC
println("Area under ROC = " + auROC)

Threshold: 1.0, Precision: 1.0
Threshold: 0.0, Precision: 0.17782909930715934
Threshold: 1.0, Recall: 0.03896103896103896
Threshold: 0.0, Recall: 1.0
Threshold: 1.0, F-score: 0.07500000000000001, Beta = 1
Threshold: 0.0, F-score: 0.30196078431372547, Beta = 1
Threshold: 1.0, F-score: 0.07500000000000001, Beta = 0.5
Threshold: 0.0, F-score: 0.30196078431372547, Beta = 0.5
Area under precision-recall curve = 0.6049308659008429
Area under ROC = 0.5194805194805194


precision: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[490] at map at BinaryClassificationMetrics.scala:236
recall: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[491] at map at BinaryClassificationMetrics.scala:236
PRC: org.apache.spark.rdd.RDD[(Double, Double)] = UnionRDD[494] at union at BinaryClassificationMetrics.scala:127
f1Score: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[495] at map at BinaryClassificationMetrics.scala:236
beta: Double = 0.5
fScore: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[496] at map at BinaryClassificationMetrics.scala:236
auPRC: Double = 0.6049308659008429
thresholds: org.apache.spark.rdd.RDD[Double] = MapPartitionsRDD[501] at map at <console>:74
roc: org.apache.spark.rdd.RDD[(Double, Do...


In [24]:
val df = spark.createDataFrame(Seq(
  (0.0, 1.0),
  (1.0, 0.0),
  (2.0, 1.0),
  (0.0, 2.0),
  (0.0, 1.0),
  (2.0, 0.0)
)).toDF("categoryIndex1", "categoryIndex2")

val encoder = new OneHotEncoder()
  .setInputCols(Array("categoryIndex1", "categoryIndex2"))
  .setOutputCols(Array("categoryVec1", "categoryVec2"))
val model = encoder.fit(df)

val encoded = model.transform(df)
encoded.show()

+--------------+--------------+-------------+-------------+
|categoryIndex1|categoryIndex2| categoryVec1| categoryVec2|
+--------------+--------------+-------------+-------------+
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           1.0|           0.0|(2,[1],[1.0])|(2,[0],[1.0])|
|           2.0|           1.0|    (2,[],[])|(2,[1],[1.0])|
|           0.0|           2.0|(2,[0],[1.0])|    (2,[],[])|
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           2.0|           0.0|    (2,[],[])|(2,[0],[1.0])|
+--------------+--------------+-------------+-------------+



df: org.apache.spark.sql.DataFrame = [categoryIndex1: double, categoryIndex2: double]
encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_c94a547e62ef
model: org.apache.spark.ml.feature.OneHotEncoderModel = OneHotEncoderModel: uid=oneHotEncoder_c94a547e62ef, dropLast=true, handleInvalid=error, numInputCols=2, numOutputCols=2
encoded: org.apache.spark.sql.DataFrame = [categoryIndex1: double, categoryIndex2: double ... 2 more fields]
